### 1. node degree가 중간값인 node를 찾기 (tie brake: index 작은 것 우선)

In [5]:
!python gat.py --problem 1

degree of nodes: tensor([3., 3., 5.,  ..., 1., 4., 4.])
median degree: 3.0
median degree node index: 0


### 2. 위 node의 link를 찾고, 각각 attention coefficient를 찾기

In [23]:
!python gat.py --problem 2 --no-log

median edges: tensor([ 633, 1862, 2582,    0])
attention coefficient: tensor([0.2446, 0.1971, 0.2508, 0.2502])


c:\Users\choi\Graph-machine-learning\gat.py:142: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  median_node = torch.tensor(median_node, device=data.edge_index.device)


### 3. GCN과의 node classification을 비교

In [ ]:
!python gcn.py --problem 8 --no-log

In [ ]:
!python gat.py --problem 3 --no-log

### 4. $\vec{\mathbf{a}} \in \mathbb{R}^{2F'}$일 경우 source와 target이 서로 다른 weight를 가지는지 설명

$W$는 같은 것을 사용하지만, $\vec{\mathbf{a}}$는 target이 $\mathbb{R}^{F'}$ 크기의 앞부분을 source가 $\mathbb{R}^{F'}$ 크기의 뒷부분을 적용하게 된다. 

### 5. 위의 문제가 코드에서 어떻게 구현되어 있는지 4번과 비교하여 설명

### 6. 코드에서 weight가 달랐다면, 같게 할때 성능이 달라지는지 확인

In [ ]:
!python gat.py --problem 6

### 7. dropout layer가 어디에 적용되는지와 그 역할을 서술

### 8. hyperparameter를 조절하여 최고의 성능을 찾기

### C. PubMed에서 최고의 성적을 내기